In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from pymongo import MongoClient
from random import uniform

import regex as re
import os

In [2]:
# Website url and the list of cities
url = "https://www.getyourguide.com"
cities = ["San Francisco", "Los Angeles", "San Diego", "Seattle", "Las Vegas", "Honolulu", "Miami", "Orlando", "Chicago", "New Orleans", "Boston", "New York City", "Washington, DC", "Nashville", "Portland"]

In [5]:
# Connect to ChromeDriver
driver = webdriver.Chrome(executable_path='chromedriver_win32\chromedriver.exe')
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

# Search for the city and save the first three pages
for city in cities:
    driver.get(url)
    sleep(2)
    # Search for the city
    inp = driver.find_element(By.CSS_SELECTOR, "input[type=text]")
    inp.send_keys(city+"\n")
    sleep(2)
    # Retrieve the frist 3 pages - contains the top 60 recommended activities
    for page in [1,2,3]:
        with open (f"city_pages/{city}_{page}.html", "w", encoding = "utf-8") as file:
            file.write(driver.page_source)
        sleep(5)
        next_page = driver.find_element(By.CSS_SELECTOR, "button.trip-item-pagination__controls-item.next.c-button.c-button--medium.c-button--outlined-standard")
        next_page.click()
        sleep(3)
    
driver.quit()

C:\Users\59699\AppData\Local\Temp\ipykernel_11988\64838709.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver_win32\chromedriver.exe')


In [108]:
# Connect to MongoDB
client = MongoClient('localhost', 27017)

# Create the database and the collection
db = client["getyourguide"]
collection = db["top_activities"]
# Drop the collection if exists
collection.drop()
collection = db["top_activities"]

In [109]:
# Loop through the saved pages for each city
for city in cities:
    rank = 0
    for page in [1,2,3]:
        with open(f"city_pages/{city}_{page}.html", "r", encoding = "utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")
            top_activities = soup.select("div.trip-item-activities-desktop__activities-container__list div.vertical-activity-card__content-wrapper")
            for i in top_activities:
                rank += 1
                # Specify the city and the rank of the activity of the city
                doc = {"city": city, "rank": rank}
                
                # Extract the name of the activity
                act_name = i.find("p", {"class":"vertical-activity-card__title"}).text.strip()
                doc["name"] = act_name
                filename = re.findall("[A-Za-z0-9 ,&-]+", act_name)[-1].strip()
                
                # Extract the category of the activity
                doc["category"] = i.find("span", {"class":"vertical-activity-card__activity-type"}).text.strip()
                
                # Extract the link to the activity page
                link = i.find("a")['href']
                if page == 1:
                    act_url = url+link
                else:
                    act_url = link
                doc["link"] = act_url

                # Extract the number of reviews if any
                try:
                    review = i.find("div", {"class":"rating-overall__reviews"}).text.strip()
                    review = re.findall("[0-9]+", review)
                    review = "".join(review)
                    review = int(review)
                    doc["reviews"] = review
                except:
                    pass
                
                # Extract the rating if any
                try:
                    rating = i.find("span", {"class":"rating-overall__rating-number rating-overall__rating-number--right"}).text.strip()
                    rating = float(rating)
                    doc["rating"] = rating
                except:
                    pass

                # Extract the pricing and the pricing category if any
                try:
                    pricing = i.find("div", {"class":"baseline-pricing__value"}).text 
                    pricing = re.findall("([0-9.]+)", pricing)[0]
                    pricing = float(pricing)
                    pricing_category = i.find("p", {"class":"baseline-pricing__category"}).text.strip()
                    doc["pricing"] = pricing
                    doc["pricing_category"] = pricing_category
                except:
                    pass
                               
                # Store all the information to MongoDB
                collection.insert_one(doc)

In [60]:
# Subset the stored data
cursor = collection.find({})
data = {'city':[], 'rank':[], 'name':[], "link":[]}
for i in cursor:
    data['city'].append(i['city'])
    data['rank'].append(i['rank'])
    data['name'].append(i['name'])
    data['link'].append(i['link'])
client.close()

In [59]:
# Connect to ChromeDriver
driver = webdriver.Chrome(executable_path='chromedriver_win32\chromedriver.exe')
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

# Save the activity page based on the link extracted
for i in range(len(data['city'])):
    city = data['city'][i]
    rank = data['rank'][i]
    name = data['name'][i]
    link = data['link'][i]
    filename = re.findall("[A-Za-z0-9 ,&-]+", name)[-1].strip()
    
    driver.get(link)
    sleep(2)
    scroll_down = uniform(600,1200)
    driver.execute_script(f"window.scrollTo(0,{scroll_down})")
    with open (f"top_activities/{city}/{str(rank).zfill(2)}_{filename}.html", "w", encoding = "utf-8") as file:
        file.write(driver.page_source)
    sleep(round(uniform(5,9),1))

driver.quit()

C:\Users\59699\AppData\Local\Temp\ipykernel_11988\1057542059.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver_win32\chromedriver.exe')


In [94]:
# Connect to MongoDB
client = MongoClient('localhost', 27017)

# Access the database and the collection
db = client["getyourguide"]
collection = db["top_activities"]

In [110]:
# Loop through each acticity page to get details and Update the details to the collection
for city in cities:
    for filename in os.listdir(f"top_activities/{city}"):
        try:
            with open(f"top_activities/{city}/{filename}", "r", encoding = "utf-8") as file:
                doc = {}
                rank = int(filename[:2])
                soup = BeautifulSoup(file, "html.parser")
                # return a list of details
                details = soup.select("section.activity-key-details__container div")
                for i in details:
                    try:
                        detail_cat = i.select("dt span:nth-child(2)")[0].text
                        detail_info = i.find("dd").text.strip()

                        # Combine "Free cancellation" with "Cancellation policy" categories
                        if "cancellation" in detail_cat:
                            detail_cat = "Cancellation policy"

                        # Replace "Reserve now & pay later" with "Reservation policy" for more legibleness
                        if "Reserve now" in detail_cat:
                            detail_info = detail_cat
                            detail_cat = "Reservation policy"
                                      
                        # Combine details about durations
                        if "Duration" in detail_cat:
                            detail_info = re.findall("Duration (.*)", detail_cat)[0]
                            detail_cat = "Duration"
                        
                        # Combine details about valid time
                        if "Valid" in detail_cat:
                            detail_info = re.findall("Valid (.*)", detail_cat)[0]
                            detail_cat = "Valid for"
                            
                        # Combine details about "skip the line"
                        if "Skip the line through" in detail_cat:
                            detail_info = detail_cat
                            detail_cat = "Skip the ticket line"
                            
                        # Combine details about small group and private group
                        if "group" in detail_cat:
                            detail_cat = "Private group or small groups available"
                            
                        # Replace "Instructor" with "Host or greeter"
                        if "Instructor" in detail_cat:
                            detail_cat = "Host or greeter"

                        # Point out available services
                        if detail_info == "":
                            detail_info = "Yes"
                    except:
                        continue
                    doc[detail_cat] = detail_info
            collection.update_one({"rank":rank, "city":city}, {"$set":doc})
                
        except:
            continue
client.close()